In [2751]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE

In [2752]:
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2753]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error

In [2754]:
data = pd.read_csv('https://github.com/AnonymousCITK/ProjectML/raw/main/heart.csv')
# print(data.shape)
data.head(100)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,45,0,0,138,236,0,0,152,1,0.2,1,0,2,1
96,57,0,0,128,303,0,0,159,0,0.0,2,1,2,1
97,53,1,0,123,282,0,1,95,1,2.0,1,2,3,0
98,65,1,0,110,248,0,0,158,0,0.6,2,2,1,0


#[*] Plot the data

In [2755]:
# plt.scatter(data.age, data.currentSmoker, marker='*', color='green')
# plt.scatter(data.age, data.diabetes, marker='*', color='blue')

In [2756]:
# plt.hist(data, histtype='step')
# data.hist()
# plt.show()

In [2757]:
print(data.columns)
data.info()

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [2758]:
print("Original: \n",data.describe())

Original: 
                age          sex           cp     trestbps        chol  \
count  1025.000000  1025.000000  1025.000000  1025.000000  1025.00000   
mean     54.434146     0.695610     0.942439   131.611707   246.00000   
std       9.072290     0.460373     1.029641    17.516718    51.59251   
min      29.000000     0.000000     0.000000    94.000000   126.00000   
25%      48.000000     0.000000     0.000000   120.000000   211.00000   
50%      56.000000     1.000000     1.000000   130.000000   240.00000   
75%      61.000000     1.000000     2.000000   140.000000   275.00000   
max      77.000000     1.000000     3.000000   200.000000   564.00000   

               fbs      restecg      thalach        exang      oldpeak  \
count  1025.000000  1025.000000  1025.000000  1025.000000  1025.000000   
mean      0.149268     0.529756   149.114146     0.336585     1.071512   
std       0.356527     0.527878    23.005724     0.472772     1.175053   
min       0.000000     0.000000   

#[*] Detailed observation from the data:

In [2759]:
#Checking if there is any missing values
print(data.isnull().sum())
# print(data2)

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [2760]:
#grouping the mean of the features of the data according to the target value, here it (0 or 1) or (Well or Unwell)
data.groupby('target').mean()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
target,,,,,,,,,,,,,
0,56.569138,0.827655,0.482966,134.106212,251.292585,0.164329,0.456914,139.130261,0.549098,1.600200,1.166333,1.158317,2.539078
1,52.408745,0.570342,1.378327,129.245247,240.979087,0.134981,0.598859,158.585551,0.134981,0.569962,1.593156,0.370722,2.119772


#[*] Assuming the target variable is 'Heart_ stroke'
#[*] Here, 0 --> Good Heart and 1 --> Bad Heart


In [2761]:
data['target'].value_counts()

1    526
0    499
Name: target, dtype: int64

##[*] When a column is droped, axix = 1
##[*] while dropping a row the, axis = 0
##[*] drop() method  

#[*] sklearn dependencies:

In [2762]:
#removing all the rows containing missing values and storing it to cln_data variable
# data = data.dropna()
# print(data)
data.describe()
print(data.isnull().sum())

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [2763]:
#Spiting the Features and the target:
X = data.drop(columns = 'target', axis=1)
Y = data['target']

# print(X)
# print(Y)

In [2764]:
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# # print(X_scaled)
# X = X_scaled
# Y = data['target']

In [2765]:
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X) , columns = X.columns)
Y = data['target']

In [2766]:
# Splitting the Data into Training and Test data
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.2, stratify=Y, random_state=42)

# y_train.value_counts().plot.pie(autopct = '%.2f')

In [2767]:
print(X.shape, x_train.shape, x_test.shape)

(1025, 13) (205, 13) (820, 13)


In [2768]:
# sm = SMOTE(random_state=0)
# x_train , y_train = sm.fit_resample(x_train , y_train)

#[*] Model Training:

In [2769]:
# model = svm.SVC(kernel='linear')
model = LogisticRegression()

In [2770]:
model.fit(x_train, y_train)

LogisticRegression()

In [2771]:
pdt = model.predict(X)

#[*] Acurracy for Training data

In [2772]:
model.score(X,Y)

0.824390243902439

In [2773]:
train_pred = model.predict(x_train)
train_accuracy = accuracy_score(train_pred, y_train)
print("Training Accuracy:", train_accuracy)

Training Accuracy: 0.8341463414634146


#[*] Acurracy for Testing data

In [2774]:
test_pred = model.predict(x_test)
test_accuracy = accuracy_score(y_test, test_pred)
print("Testing Accuracy:", test_accuracy)

Testing Accuracy: 0.8219512195121951


In [2775]:
mean_squared_error(y_test, test_pred)


0.17804878048780487

In [2776]:
confusion_matrix(y_test, test_pred)

array([[306,  93],
       [ 53, 368]])

In [2777]:
# Assuming 'y_test' contains the actual labels and 'y_pred' contains the predicted labels

precision = precision_score(y_test, test_pred)
recall = recall_score(y_test, test_pred)
f1 = f1_score(y_test, test_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.7982646420824295
Recall: 0.8741092636579573
F1 Score: 0.8344671201814059


#[*] Prediptic system

In [2778]:
DATA = [43,0,2,122,213,0,1,165,0,0.2,1,0,2]

In [2779]:
print(DATA)


[43, 0, 2, 122, 213, 0, 1, 165, 0, 0.2, 1, 0, 2]


In [2780]:
DATA_as_np_array = np.asarray(DATA)

#reshaping the array as prediction is only for 1 instance
DATA_reshape = DATA_as_np_array.reshape(1,-1)

std_DATA = scaler.transform(DATA_reshape)
# print(std_DATA)

pred = model.predict(std_DATA)
print(pred)



if (pred[0] == 0):
  print("The person does not have heart disease.")
else:
  print("The person has heart disease.")

[1]
The person has heart disease.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [2781]:
# print(metrics.classification_report(y, pdt))